In [ ]:
Mantener todas las features puede ser válido, pero con matices.

Decisión razonable: sí, si usas modelos robustos a multicolinealidad (árboles/ensembles como RandomForest, Gradient Boosting, XGBoost) y validas con un split temporal. Es común mantener redundancias al inicio y dejar que el modelo “ignore” las débiles.
Riesgos principales: coeficientes inestables en modelos lineales, interpretabilidad más difícil, y mayor costo computacional. En tus resultados hay pares casi duplicados (|r|≈1), lo que afectaría especialmente a regresiones/logísticas sin regularización.
Excepción crítica: columnas 100% faltantes. similitud_promedio_vecinos y conflicto_modelos están totalmente NaN; así no se pueden entrenar modelos de scikit-learn. Si no quieres “eliminarlas” del dataset maestro, al menos exclúyelas al construir la matriz de entrenamiento o imputa un constante sabiendo que no aportarán señal.
Si decides mantener todas

Modelo: prioriza RandomForest/GradientBoosting/XGBoost (menos sensibles a colinealidad). Para lineales, usa regularización (Ridge/LogisticRegression con penalty='l2' o elasticnet) y estandariza.
Validación: usa validación temporal y compara métricas con y sin recorte (no necesitas borrar columnas del CSV, sólo excluir al entrenar).
Interpretación: confía más en permutation importance que en coeficientes cuando hay colinealidad. Es normal que L1 deje muchos coeficientes en 0 o elija arbitrariamente entre correlacionadas.
Categorías problemáticas: evita usar id_sismo_principal y Date(UTC) crudos en el modelo; si quieres conservarlos en el dataset, no los alimentes al modelo (o transforma fecha a señales útiles como mes, hora, estacionalidad).
Cuándo sí recortar (mínimo)

Columnas 100% NaN: exclúyelas al entrenar hasta implementarlas de verdad.
Duplicados exactos/derivados triviales (p. ej., magnitud_umbral duplicando Magnitude): no aportan y sólo aumentan ruido/tiempo. Puedes conservar en el dataset, pero no pasarlas al modelo.
En resumen: es aceptable no eliminar features del dataset maestro. Para el entrenamiento, aplica filtros “no destructivos” (excluir al vuelo) de: todas-NaN, identificadores/fechas crudas y, si usas modelos lineales, tratar la colinealidad con regularización